In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "OneTwoThree"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    #html image tag
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    #identifier
    html.Center(html.H1('Created by Benjamin Abbott')),
    html.Hr(),
    html.Div(
        #Radio buttons to filter types
        dcc.RadioItems(
        id='filter-type',
        #labels
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
            {'label': 'Reset', 'value': 'RESET'},
        ],
        value='RESET',
        labelStyle={'display':'inline-block'}    
        )  
    ),
    
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        data=df.to_dict('records'),
        
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False, 
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
               
def update_dashboard(value):
# Filter interactive data table with MongoDB queries

    if value == 'WR':
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [
                                                            {'animal_type':'Dog'},
                                                            {'sex_upon_outcome': 'Intact Female'},
                                                            {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}},            
                                                            {'$or': [
                                                                {'breed':'Chesa Bay Retr'},
                                                                {'breed':'Labrador Retriever Mix'},
                                                                {'breed':'Newfoundland Mix'},                                                               
                                                            ]}]}))

    elif value == 'MWR':
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [
                                                            {'sex_upon_outcome': 'Intact Male'},
                                                            {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}},            
                                                            {'$or': [
                                                                {'breed':{'$regex':'German Shepherd'}},
                                                                {'breed':{'$regex':'Alaskan Malamute'}},
                                                                {'breed':{'$regex':'Old English Sheepdog'}}, 
                                                                {'breed':{'$regex':'Siberian Husky'}},
                                                                {'breed':{'$regex':'Rottweiler'}},                                                                
                                                            ]}]}))
    elif value == 'DIT':
        df = pd.DataFrame.from_records(shelter.read_all({'$and': [
                                                            {'sex_upon_outcome': 'Intact Male'},
                                                            {'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 300}},            
                                                            {'$or': [
                                                                {'breed':{'$regex':'Doberman Pinscher'}},
                                                                {'breed':{'$regex':'German Shepherd'}},
                                                                {'breed':{'$regex':'Golden Retriever'}}, 
                                                                {'breed':{'$regex':'Bloodhound'}},
                                                                {'breed':{'$regex':'Rottweiler'}},                                                                
                                                            ]}]})) 

    elif value == 'RESET':
        df = pd.DataFrame.from_records(shelter.read_all({}))
        
    
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]     
    data=df.to_dict('records')
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    dff = dff['breed'].value_counts().reset_index()
    dff.columns = ['breedNames', 'count']
    return [
       dcc.Graph(
           id='graph_id',
           figure = px.pie(dff, 
                           values='count', 
                           names='breedNames',
                           title='Total Age of Particular Breeds'  
                          ),
       )]   
                                
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ]) 
        ])
    ]


app